In [22]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl
import scipy
import corner
import pymc3 as pm
import theano
import theano.tensor as T
import seaborn as sns
import os
import sys
sys.path.append('../codebase')
from data_preprocessing_ogle import process_data
from plotting_utils import plot_data

mpl.rc('font',**{'family':'serif','serif':['Palatino']})
mpl.rc('text', usetex=False)
mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['xtick.labelsize'] = 20
mpl.rcParams['ytick.labelsize'] = 20
mpl.rcParams['axes.titlesize'] = 16

events = [] # event names
lightcurves = [] # data for each event
 
i = 0
n_events = 100
max_length = 0
lengths = np.zeros(n_events, dtype=int)

for entry in os.scandir('../../../../data/OGLE_ews/2017/'):
    if entry.is_dir() and (i < n_events):
        events.append(entry.name)
        photometry = np.genfromtxt(entry.path + '/phot.dat', usecols=(0,1,2))
        lightcurves.append(photometry)
        if (len(photometry) > max_length):
            max_length = len(photometry)
        lengths[i] = len(photometry)
        i = i + 1
        
print("Loaded events:", events)
print("Max length: ", max_length)

Loaded events: ['blg-0305', 'blg-0089', 'blg-1403', 'blg-0642', 'blg-0717', 'blg-0055', 'blg-0058', 'blg-1507', 'blg-1261', 'blg-1390', 'blg-1159', 'blg-0444', 'blg-0919', 'blg-1273', 'blg-1360', 'blg-0931', 'blg-1660', 'blg-1183', 'blg-0047', 'blg-0237', 'blg-0024', 'blg-0327', 'blg-1114', 'blg-0193', 'blg-0559', 'blg-0885', 'blg-0508', 'blg-0370', 'blg-0154', 'blg-0899', 'blg-0683', 'blg-1346', 'blg-0577', 'blg-0690', 'blg-0078', 'blg-1544', 'blg-1776', 'blg-0405', 'blg-1647', 'blg-0852', 'blg-0388', 'blg-0383', 'blg-1109', 'blg-0599', 'blg-1097', 'blg-1072', 'blg-0113', 'blg-0826', 'blg-0873', 'blg-1536', 'blg-1746', 'blg-0729', 'blg-1702', 'blg-1549', 'blg-1060', 'blg-0726', 'blg-1774', 'blg-0403', 'blg-1666', 'blg-1468', 'blg-0967', 'blg-0432', 'blg-1768', 'blg-0828', 'blg-0167', 'blg-1620', 'blg-0814', 'blg-0028', 'blg-1220', 'blg-0260', 'blg-1440', 'blg-1327', 'blg-0580', 'blg-0784', 'blg-0076', 'blg-1673', 'blg-0141', 'blg-0864', 'blg-0358', 'blg-0155', 'blg-0734', 'blg-1691', 

In [24]:
# Collect all lightcurves into a single large numpy array
lcrvs_t = np.zeros((len(lightcurves), max_length))
lcrvs_F = np.zeros((len(lightcurves), max_length))
lcrvs_sigF = np.zeros((len(lightcurves), max_length))

for i, lc in enumerate(lightcurves):
    lcrvs_t[i, :len(lightcurves[i][:, 0])] = lightcurves[i][:, 0]
    lcrvs_F[i, :len(lightcurves[i][:, 0])] = lightcurves[i][:, 1]
    lcrvs_sigF[i, :len(lightcurves[i][:, 0])] = lightcurves[i][:, 2]
    
# Convert numpy arrays to theano tensors
lcrvs_t = T.cast(lightcurves_t, 'float64')
lcrvs_F = T.cast(lightcurves_F, 'float64')
lcrvs_sigF = T.cast(lightcurves_sigF, 'float64')
lengths = T.cast(lengths, 'int32')

In [26]:
model = pm.Model()

with model:    
    n_population = len(lightcurves)
    
    BoundedNormal = pm.Bound(pm.Normal, lower=0.0) # Half-Normal dist

    # Hyperpriors
    mu_tE = BoundedNormal('mu_tE', mu=100, sd=200.)
    sig_tE = BoundedNormal('sig_tE', mu=20, sd=100.)
        
    # Priors 
    tE = pm.Lognormal('tE', mu=mu_tE, sd=sig_tE, shape=n_population)
    DeltaF = BoundedNormal('DeltaF', mu=[np.max(lc[:, 1]) for lc in lightcurves], 
                           sd=1., shape=n_population)
    Fb = pm.Normal('Fb', mu=0., sd=0.1, shape=n_population)
    t0 = pm.Uniform('t0', [np.min(lc[:, 0]) for lc in lightcurves], 
                    [np.max(lc[:, 0]) for lc in lightcurves], shape=n_population)
    # p(teff|tE)
    def conditional_prior(value):
        teff = T.cast(value, 'float64')
        return -T.log(tE) - (teff/tE)**2/1**2 
    
    teff = pm.DensityDist('teff', conditional_prior, shape=n_population)
    
    # Likelihood 
    def ln_likelihood(lightcurves):
        lnL = T.cast(0., 'float64')
        
        # We can't just write a for loop, we have to use Theano scan
        
        n = T.iscalar("n")
        mu_F = T.isvector("mu_F")
        
        def ln_likelihood_single(n):
            lightcurve = T.cast(lightcurves[n], 'float64')
            t = lightcurve[:, 0]
            F = lightcurve[:, 1]
            sig_F = lightcurve[:, 2]
        
            # Mean model
            u0 = teff/tE
            u = T.sqrt(u0[n]**2 + ((t - t0[n])/tE[n])**2)
            A = lambda u: (u**2 + 2)/(u*T.sqrt(u**2 + 4))

            mu_F = DeltaF[n]*(A(u) - 1)/(A(u0[n]) - 1) + Fb[n]
            
            # Gaussian likelihood
            return T.sum((-(mu_F - F)**2/sig_F**2))
        
        
        # Symbolic description of the result
        result, updates = theano.scan(fn=ln_likelihood_single,
                              n_steps=n_population,
                              outputs_info=T.ones_like(n_pop),)
        
        return lnL
    
    likelihood = pm.DensityDist('likelihood', ln_likelihood, observed={'lightcurves':lightcurves})

model.check_test_point()
model.vars

ValueError: Input dimension mis-match. (input[0].shape[1] = 4530, input[1].shape[1] = 100)

In [ ]:
pm.model_to_graphviz(model1)